In [8]:
from mrcnn.utils import compute_overlaps
import mrcnn.config as config
import mrcnn.model as Model
from Visualize import DisplayInstances,Park
import os
import cv2

In [2]:
#defining paths
ROOT_DIR=os.path.abspath("/home/noor/projects/car parking/")

#Directory to save trained model
MODEL_DIR=os.path.join(ROOT_DIR,"model")

#Coco weights file path
WEIGHT_DIR=os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")

In [3]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [4]:
#Inherit from base class Config
class Inference(config.Config):
    #batch_size=GPU_COUNT*IMAGES_PER_GPU
    #GPU_COUNT=1
      NAME="PARK"
      IMAGES_PER_GPU= 1
      NUM_CLASSES=80+1
configuration=Inference()
configuration.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
#create inference(testing) model
Model=Model.MaskRCNN(mode="inference",config=configuration,model_dir=MODEL_DIR)

#load pretrained weights for the model
Model.load_weights(filepath=WEIGHT_DIR,by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
def main(record=False,path=None):

    first_name=True
    frames=[]
    detect_count=0
    space_count=0
    frame_count=0

    cap=cv2.VideoCapture("parking.mp4")
    while (cap.isOpened()):
        ret,frame=cap.read()#read frame
        frame_count +=1
        if ret:
            output=Model.detect(images=[frame],verbose=0)#a list of dictionaries(one dict per image)
            output=output[0]
            box=output["rois"]#bounding boxes of objects
            image=DisplayInstances(frame,output["rois"],output["masks"],output["class_ids"],class_names,output["scores"])
            frames.append(image)
            if first_name:
                box1=box
                first_name=False
                continue
        
            if detect_count < 5:#observe first 5 frames for parking(cars in parking) detection 
                box1=Park(box1,box)#filter objects in box1
                detect_count+=1 
                print("yes")
            else:
                overlaps=utils.compute_overlaps(box1,box)
                detect_count+=1
                overlap_count=sum((sum(i > 0.4  for i in row)) for row in overlaps)
                if overlap_count < len(box1):
                    space_count +=1
                
                    if frame_count==space_count:#check "n" consecutive frames 
                        if space_count==10:
                            twilio()#message the user
                            break   
                    else:
                        space_count=0
                        frame_count=0   
        else:
            break
                
    if record:            
        #create a video writer object
        fourcc=cv2.VideoWriter_fourcc(*'XVID')
        video=cv2.VideoWriter(path,fourcc,10,(450,360))
                
        for i in range(len(frames)):
            video.write(frames[i])#write video in video instance

    video.release()
    cap.release()    

In [ ]:
main(record=True,path="check.mp4")